In [ ]:
import pandas as pd

def compare_players(player_names, file_path="Fantasy Data.xlsx"):
    """
    Returns two DataFrames:
      1) consistency_sorted: grouped by 2024 AVG PPG tiers (labels "7".."1"),
         then ordered within tier by 2024 Consistency Score (desc).
      2) maxppg_sorted: grouped by 2023-2024 Max PPG tiers (labels "7".."1"),
         then ordered within tier by 2024 TTL PPG (desc).

    Notes on MaxPPG tiers:
      - To keep exactly 7 tiers with your labels, values <16.5 are folded into tier "7",
        and values >49.5 are folded into tier "1".
    """

    df = pd.read_excel(file_path, sheet_name="Draft Projections")

    # Keep only players of interest + needed columns
    cols = [
        "PLAYER NAME",
        "2024 AVG PPG",
        "2024 TTL PPG",
        "AVG Games Missed",
        "2024 Games Missed",
        "2024 Consistency Score",
        "2024 Team Point Share %",
        "2023-2024 Max PPG",
    ]
    data = df[df["PLAYER NAME"].isin(player_names)][cols].copy()

    # -------------------------
    # Consistency table (PPG tiers)
    # -------------------------
    ppg_bins = [7.7, 10.4, 13.1, 15.8, 18.5, 21.2, 23.9, 26.6]
    ppg_labels = ["7", "6", "5", "4", "3", "2", "1"]  # 7 bins -> 7 labels

    data["PPG Tier"] = pd.cut(
        data["2024 AVG PPG"],
        bins=ppg_bins,
        labels=ppg_labels,
        include_lowest=True
    )
    data["PPG Tier"] = pd.Categorical(data["PPG Tier"], categories=ppg_labels, ordered=True)

    consistency_sorted = (
        data.sort_values(by=["PPG Tier", "2024 Consistency Score"], ascending=[False, False])
            .loc[:, ["PLAYER NAME", "2024 AVG PPG", "2024 TTL PPG",
                     "AVG Games Missed", "2024 Consistency Score",
                     "2024 Team Point Share %", "PPG Tier"]]
            .reset_index(drop=True)
    )

    # -------------------------
    # MaxPPG table (Max PPG tiers)
    # -------------------------
    maxppg_bins = [16.5, 22, 27.5, 33, 38.5, 44, 49.5]
    maxppg_labels = ["7", "6", "5", "4", "3", "2", "1"]  # exactly 7 tiers

    # Fold extremes into the nearest tier to maintain 7 labels
    def maxppg_tier(val):
        if pd.isna(val):
            return pd.NA
        if val <= maxppg_bins[0]:       # <= 16.5
            return "7"
        elif val <= maxppg_bins[1]:     # 16.5 - 22
            return "6"
        elif val <= maxppg_bins[2]:     # 22 - 27.5
            return "5"
        elif val <= maxppg_bins[3]:     # 27.5 - 33
            return "4"
        elif val <= maxppg_bins[4]:     # 33 - 38.5
            return "3"
        elif val <= maxppg_bins[5]:     # 38.5 - 44
            return "2"
        elif val <= maxppg_bins[6]:     # 44 - 49.5
            return "1"
        else:                           # > 49.5
            return "1"

    data["MaxPPG Tier"] = data["2023-2024 Max PPG"].apply(maxppg_tier)
    data["MaxPPG Tier"] = pd.Categorical(data["MaxPPG Tier"],
                                         categories=maxppg_labels, ordered=True)

    maxppg_sorted = (
        data.sort_values(by=["MaxPPG Tier", "2024 TTL PPG"], ascending=[False, False])
            .loc[:, ["PLAYER NAME", "2024 AVG PPG", "2024 TTL PPG",
                     "2023-2024 Max PPG", "2024 Games Missed",
                     "2024 Team Point Share %", "MaxPPG Tier"]]
            .reset_index(drop=True)
    )

    return consistency_sorted.to_string(index=False), maxppg_sorted.to_string(index=False)


In [ ]:
# Usage for RB:
rbs_to_compare = ["Breece Hall", "Alvin Kamara"]
consistency_table, maxppg_table = compare_players(rbs_to_compare)
print(consistency_table)
print()
print(maxppg_table)

 PLAYER NAME  2024 AVG PPG  2024 TTL PPG  AVG Games Missed  2024 Consistency Score  2024 Team Point Share % PPG Tier
Alvin Kamara          18.9         265.3          2.166667                    2.17                    18.21        3
 Breece Hall          14.8         222.7          3.333333                    1.80                    14.94        5

 PLAYER NAME  2024 AVG PPG  2024 TTL PPG  2023-2024 Max PPG  2024 Games Missed  2024 Team Point Share % MaxPPG Tier
Alvin Kamara          18.9         265.3               44.0                2.0                    18.21           2
 Breece Hall          14.8         222.7               43.1                1.0                    14.94           2


In [ ]:
# TE usage:
tes_to_compare = ["Kyle Pitts", "Dalton Kincaid", "Jake Ferguson", "Tucker Kraft"]
consistency_table, maxppg_table = compare_players(tes_to_compare)
print(consistency_table)
print()
print(maxppg_table)

   PLAYER NAME  2024 AVG PPG  2024 TTL PPG  AVG Games Missed  2024 Consistency Score  2024 Team Point Share % PPG Tier
Dalton Kincaid           7.7         100.8               2.0                    2.26                     5.66        7
 Jake Ferguson           7.7         101.2               3.0                    1.67                     6.69        7
  Tucker Kraft           9.7         155.7               1.5                    1.51                    10.21        7

   PLAYER NAME  2024 AVG PPG  2024 TTL PPG  2023-2024 Max PPG  2024 Games Missed  2024 Team Point Share % MaxPPG Tier
  Tucker Kraft           9.7         155.7               24.8                0.0                    10.21           5
 Jake Ferguson           7.7         101.2               22.1                3.0                     6.69           5
Dalton Kincaid           7.7         100.8               17.5                3.0                     5.66           6


In [ ]:
# WR Usage:
wrs_to_compare = ["Adam Thielen", "Michael Pittman Jr.", "Cedric Tillman"]
consistency_table, maxppg_table = compare_players(wrs_to_compare)
print(consistency_table)
print()
print(maxppg_table)

        PLAYER NAME  2024 AVG PPG  2024 TTL PPG  AVG Games Missed  2024 Consistency Score  2024 Team Point Share % PPG Tier
       Adam Thielen          14.4         130.1          3.333333                    1.82                    10.81        5
Michael Pittman Jr.          10.1         152.6          1.200000                    1.62                    11.64        7
     Cedric Tillman           7.3          80.4          5.000000                    0.72                     6.14      NaN

        PLAYER NAME  2024 AVG PPG  2024 TTL PPG  2023-2024 Max PPG  2024 Games Missed  2024 Team Point Share % MaxPPG Tier
       Adam Thielen          14.4         130.1               31.5                7.0                    10.81           4
     Cedric Tillman           7.3          80.4               28.9                5.0                     6.14           4
Michael Pittman Jr.          10.1         152.6               27.5                1.0                    11.64           5


In [ ]:
# QB Usage:
qbs_to_compare = ["Kyler Murray", "Justin Fields", "Michael Penix Jr.", "Patrick Mahomes II", "Caleb Williams"]
consistency_table, maxppg_table = compare_players(qbs_to_compare)
print(consistency_table)
print()
print(maxppg_table)

       PLAYER NAME  2024 AVG PPG  2024 TTL PPG  AVG Games Missed  2024 Consistency Score  2024 Team Point Share % PPG Tier
Patrick Mahomes II          18.3         293.9          0.333333                    3.89                    17.36        4
      Kyler Murray          17.5         280.5          2.833333                    2.53                    18.51        4
    Caleb Williams          15.6         249.6          0.000000                    1.97                    18.48        5
     Justin Fields          12.0         120.1          4.500000                    1.11                     7.85        6
 Michael Penix Jr.           5.3          21.2         12.000000                    0.96                     2.74      NaN

       PLAYER NAME  2024 AVG PPG  2024 TTL PPG  2023-2024 Max PPG  2024 Games Missed  2024 Team Point Share % MaxPPG Tier
Patrick Mahomes II          18.3         293.9               34.9                0.0                    17.36           3
      Kyler Murra

In [ ]:
# Kicker Usage:
qbs_to_compare = ["Jake Bates", "Brandon Aubrey", "Cameron Dicker"]
consistency_table, maxppg_table = compare_players(qbs_to_compare)
print(consistency_table)
print()
print(maxppg_table)

   PLAYER NAME  2024 AVG PPG  2024 TTL PPG  AVG Games Missed  2024 Consistency Score  2024 Team Point Share % PPG Tier
Brandon Aubrey          11.1         178.0               0.0                    2.05                    11.42        6
    Jake Bates           9.5         153.0               0.0                    2.50                     7.71        7
Cameron Dicker          10.2         164.0               2.0                    2.08                    10.67        7

   PLAYER NAME  2024 AVG PPG  2024 TTL PPG  2023-2024 Max PPG  2024 Games Missed  2024 Team Point Share % MaxPPG Tier
Brandon Aubrey          11.1         178.0               22.0                0.0                    11.42           6
Cameron Dicker          10.2         164.0               21.0                0.0                    10.67           6
    Jake Bates           9.5         153.0               19.0                0.0                     7.71           6


In [ ]:
# compare consistency of players with composite score
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

def compare_consistency_composite(player_names, file_path="Fantasy Data.xlsx"):
    """
    Compare players by combining multiple consistency-related stats into one score.
    """

    df = pd.read_excel(file_path, sheet_name="Draft Projections")

    # Filter only chosen players
    relevant_cols = [
        "PLAYER NAME",
        "AVG Consistency Score",
        "TTL Consistency Score",
        "Team Point Share % Consistency Score",
        "AVG Games Missed",
        "Overall AVG PPG Consistency Score"
    ]
    filtered_df = df[df["PLAYER NAME"].isin(player_names)][relevant_cols].copy()

    # Normalize numeric columns so they are on the same scale
    scaler = MinMaxScaler()
    numeric_cols = [
        "AVG Consistency Score",
        "TTL Consistency Score",
        "Team Point Share % Consistency Score",
        "AVG Games Missed",
        "Overall AVG PPG Consistency Score"
    ]
    filtered_df[numeric_cols] = scaler.fit_transform(filtered_df[numeric_cols])

    # Create composite score (you can tweak the weights)
    filtered_df["Overall Consistency Composite"] = (
        filtered_df["AVG Consistency Score"] * 0.3 +
        filtered_df["TTL Consistency Score"] * 0.3 +
        filtered_df["Team Point Share % Consistency Score"] * 0.15 +
        (1 - filtered_df["AVG Games Missed"]) * 0.15 +  # fewer games missed = better
        filtered_df["Overall AVG PPG Consistency Score"] * 0.1
    )

    # Sort descending by composite score
    filtered_df = filtered_df.sort_values(
        by="Overall Consistency Composite", ascending=False
    ).reset_index(drop=True)

    return filtered_df.to_string(index=False)


In [ ]:
# Example compare_consistency_composite
players_to_compare = ["Jonathan Taylor", "James Cook", "Kenneth Walker III", "Alvin Kamara", "Josh Jacobs", "Saquon Barkley", "Bijan Robinson"]
results = compare_consistency_composite(players_to_compare)
print(results)
#

       PLAYER NAME  AVG Consistency Score  TTL Consistency Score  Team Point Share % Consistency Score  AVG Games Missed  Overall AVG PPG Consistency Score  Overall Consistency Composite
Kenneth Walker III               0.287356               1.000000                              1.000000          0.692308                           1.000000                       0.682361
    Bijan Robinson               1.000000               0.089921                              0.469600          0.000000                           0.294466                       0.576863
       Josh Jacobs               0.678161               0.058692                              0.195944          0.500000                           0.711294                       0.396577
      Alvin Kamara               0.425287               0.054216                              0.216758          0.500000                           0.468088                       0.298173
   Jonathan Taylor               0.448276               0.016442 

In [ ]:
# compare consistency of players through grouping
def compare_consistency_grouped(player_names, file_path="Fantasy Data.xlsx"):
    """
    Compare players by grouping:
    1. AVG Consistency Score bins
    2. Games Missed bins
    3. Sort within group by TTL Consistency Score (descending)
    """

    df = pd.read_excel(file_path, sheet_name="Draft Projections")

    # Filter only chosen players
    relevant_cols = [
        "PLAYER NAME",
        "AVG Consistency Score",
        "TTL Consistency Score",
        "Team Point Share % Consistency Score",
        "AVG Games Missed",
        "Overall AVG PPG Consistency Score"
    ]
    filtered_df = df[df["PLAYER NAME"].isin(player_names)][relevant_cols].copy()

    # Define AVG Consistency Score bins
    consistency_bins = [0, 0.98, 1.35, 1.72, 2.09, 2.46, 2.83, 3.2, 3.57, float("inf")]
    consistency_labels = [
        "9", "8", "7", "6", "5",
        "4", "3", "2", "1"
    ]
    filtered_df["Consistency Tier"] = pd.cut(
        filtered_df["AVG Consistency Score"], bins=consistency_bins, labels=consistency_labels, include_lowest=True
    )

    # Define Games Missed bins
    def games_missed_group(x):
        if x == 0:
            return "0"
        elif x <= 2:
            return "1-2"
        elif x <= 4:
            return "3-4"
        else:
            return "5+"

    filtered_df["Games Missed Tier"] = filtered_df["AVG Games Missed"].apply(games_missed_group)

    # Make categorical to ensure proper tier order
    filtered_df["Consistency Tier"] = pd.Categorical(
        filtered_df["Consistency Tier"], categories=consistency_labels, ordered=True
    )
    filtered_df["Games Missed Tier"] = pd.Categorical(
        filtered_df["Games Missed Tier"], categories=["0", "1-2", "3-4", "5+"], ordered=True
    )

    # Sort by Consistency Tier, then Games Missed Tier, then TTL Consistency Score
    filtered_df = filtered_df.sort_values(
        by=["Consistency Tier", "Games Missed Tier", "TTL Consistency Score"],
        ascending=[False, True, False]
    ).reset_index(drop=True)

    return filtered_df.to_string(index=False)


In [ ]:
# example compare_consistency_grouped
rbs_to_compare = ["Jonathan Taylor", "James Cook", "Kenneth Walker III", "Alvin Kamara", "Josh Jacobs", "Saquon Barkley", "Bijan Robinson"]
results = compare_consistency_grouped(rbs_to_compare)
print(results)

       PLAYER NAME  AVG Consistency Score  TTL Consistency Score  Team Point Share % Consistency Score  AVG Games Missed  Overall AVG PPG Consistency Score Consistency Tier Games Missed Tier
    Bijan Robinson                   2.51               5.882483                              9.587161          0.000000                           4.106181                4                 0
       Josh Jacobs                   2.23               4.495876                              5.268434          2.166667                           6.784956                5               3-4
Kenneth Walker III                   1.89              46.291074                             17.957722          3.000000                           8.640345                6               3-4
      Alvin Kamara                   2.01               4.297149                              5.596911          2.166667                           5.221976                6               3-4
   Jonathan Taylor                   2.03    